<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/test/notebooks/model-blending/blend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merging CodeLLMs to Create an Efficant Low-Memory Quantized Model for `whats-up-doc` using the TIES Method

In [ ]:
import os
import yaml
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline

## Download and Install `mergekit`

In [ ]:
dirName = "mergekit"
cwd = os.getcwd()

concatDirPath = os.path.join(cwd, dirName)

if not os.path.exists(concatDirPath):
    !git clone https://github.com/cg123/mergekit.git
    !cd mergekit && pip install -q -e .

## Login to HF with a Write API Key

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Create the YAML Config File to Merge Models with SLERP

### Write Config Script

In [ ]:
# Set the resultant model's name
MODEL_NAME = 'whats-up-llamas-ties'

MODEL_1 = "meta-llama/Meta-Llama-3-8B-Instruct"
MODEL_2 = "codellama/CodeLlama-7b-Instruct-hf"

OUTPUT_DIR = "whats-up-llamas-ties"

#### TIES YAML Config Creation

In [ ]:
yamlConfigTIESLlamas = f"""
models:
  - model: {MODEL_1}  # no parameters necessary for base model
  - model: {MODEL_2}
    parameters:
      density: 0.5
      weight: 0.5
merge_method: ties
base_model: {MODEL_1}
parameters:
  normalize: true
  int8_mask: true
dtype: float16

"""

### Save Config Script

In [ ]:
# Save the YAML configuration to a file
yamlFileName = "config.yaml"
with open(yamlFileName, "w") as f:
    f.write(yamlConfigTIESLlamas)

## Merge Models

In [ ]:
cmd = f"mergekit-yaml {yamlFileName} {OUTPUT_DIR} --allow-crimes --copy-tokenizer --out-shard-size 1B --low-cpu-memory --write-model-card --lazy-unpickle"
!{cmd}

## Get the Write Token from Kaggle Notebook Secrets

In [ ]:
from kaggle_secrets import UserSecretsClient

userSecrets = UserSecretsClient()
HF_WRITE_TOKEN = userSecrets.get_secret("HF_WRITE_TOKEN")

## Use the HF API to Write the Model to a Repository

In [ ]:
from huggingface_hub import HfApi

username = "0xVolt"

# Defined in the secrets tab in Kaggle Secrets
api = HfApi(token=HF_WRITE_TOKEN)

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

# Push the whole merged-model folder to the hub
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path=OUTPUT_DIR,
)